In [1]:
import fitz
import os
from PIL import Image
import pandas as pd
from collections import Counter
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import seaborn as sns
import plotly
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from plotly.offline import iplot, init_notebook_mode
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from IPython.display import display
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler

In [2]:
dir_path = "C:/Users/vinay/code/final year/Early_Detection_of_Diabetic_Retinopathy-main/images"
for filename in os.listdir(dir_path):
        file_path = os.path.join(dir_path, filename)
        os.remove(file_path)

In [3]:
# Assigning label names to the corresponding indexes
labels = {0: 'Mild', 1: 'Moderate', 2: 'No_DR', 3:'Proliferate_DR', 4: 'Severe'}

In [4]:
file_path = 'mild.pdf'

#Define path for saved images
images_path = 'images/'
def pdf(file_path,images_path):

#Open PDF file
    pdf_file = fitz.open(file_path)

#Get the number of pages in PDF file
    page_nums = len(pdf_file)

#Create empty list to store images information
    images_list = []

#Extract all images information from each page
    for page_num in range(page_nums):
        page_content = pdf_file[page_num]
        images_list.extend(page_content.get_images())

#Raise error if PDF has no images
    if len(images_list)==0:
        raise ValueError(f'No images found in {file_path}')

#Save all the extracted images
    for i, img in enumerate(images_list, start=1):
    #Extract the image object number
        xref = img[0]
    #Extract image
        base_image = pdf_file.extract_image(xref)
    #Store image bytes
        image_bytes = base_image['image']
    #Store image extension
        image_ext = base_image['ext']
    #Generate image file name
        image_name = str(i) + '.' + image_ext
    #Save image
        with open(os.path.join(images_path, image_name) , 'wb') as image_file:
            image_file.write(image_bytes)
            image_file.close()

In [5]:
pdf(file_path,images_path)

In [6]:
# Check if the directory is empty
if not os.listdir(dir_path):
    print("Directory is empty")
else:
    
    image = []
    # Loop through all files in the directory
    for filename in os.listdir(dir_path):
        # Check if the file is an image (you can add more extensions if needed)
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
            # Construct the full file path
            file_path = os.path.join(dir_path, filename)
            # Open the image
            img = Image.open(file_path)
            img = img.resize((256,256))
            #appending image to the image list
            image.append(img)
            

In [8]:
lm = tf.keras.models.load_model('my_model')

In [9]:
lab=[]
for img in image:
    img = np.asarray(img, dtype= np.float32)
    #normalizing the image
    img = img / 255
    #reshaping the image in to a 4D array
    img = img.reshape(-1,256,256,3)
    #making prediction of the model
    predict = lm.predict(img)
    #getting the index corresponding to the highest value in the prediction
    predict = np.argmax(predict)
    #appending the predicted class to the list
    lab.append(labels[predict])

1/1 [==============================] - 0s 62ms/step


In [10]:
counts = Counter(lab)

# Find the string with the maximum count
max_occuring_string = max(counts, key=counts.get)

# Print the maximum occurring string
print("Maximum occurring string:", max_occuring_string)

Maximum occurring string: Mild
